# Lift

 - La lift est une **métrique de performance** comme le Log-Loss, l'Accuracy, l'AUC, etc.
 - La lift est utilisée pour des problématiques de **classfication**
 - La lift c'est le **ratio** entre les exemples positifs correctement prédits et les exemples positifs réels dans l'ensemble de données du test
 - La lift est souvent utilisée en marketing
 - La lift s'utilise lorsque les **classes ne sont pas réparties uniformément**:
     - Exemple : Détection de fraude
     - Exemple : Churn

Dans l'exemple de la fraude, si l'on considère que 2% des transactions sont frauduleuses alors un modèle peut prédire que personne ne fraude, il aura tout de même une accuracy de 98% 

## Exemple du churn

 - On souhaite prédire si l'utilisateur va résilier son abonnement Netflix :
     - 1 = Classe positive
     - 0 = Classe négative

- Étapes pour déterminer la lift :
    - On classe le jeu de données selon la probabilité de churn (0 à 1)
    - On divisent le jeu de données en intervalles contenant le même nombre de données (décile : 0-0,1, 0,1-0,2, 0,2-0,3, etc.)
    - On détermine pour chaque groupe le poucentage de churn (nombre de churn / effectif total du groupe)
    
<img src="lift-chart-churn-rate.png">

\begin{equation*}
\text{lift} = \frac{TP/(TP+FP)}{(TP+FN)/(TP+TN+FP+FN)}
\end{equation*}


\begin{equation*}
\text{lift (0.9-1.0)} = \frac{Taux prédit}{Taux moyen} = \frac{0.97}{0.2} = 4,85
\end{equation*}

\begin{equation*}
\text{lift (0.8-0.9)} = \frac{Taux prédit}{Taux moyen} = \frac{0.36}{0.2} = 1.80
\end{equation*}

On a donc 4,85 fois plus de chance de retenir un utilisateur sur le départ que si on avait fait un choix aléatoire. On optimise donc le ciblage marketing des utilisateurs.

1000 lignes, 10 groupes de 100. On trace en noire le churn moyen (ici 200/1000 soit 20%)

Dans le dernier groupe (0,9 - 1,0) contenant les 100 consommateurs ayant la plus forte probabilité de churn, on constate que 97% d'entre eux ont réellement churn.


Additionally, I drew a black line for the hypothetical average churn rate (20%). This is useful to define a targeting threshold: scores below the threshold will be set to 0, scores above to 1.

This is a binary classification problem: the user either cancels the subscription (churn=1) or keeps it (churn=0).

## Code Python
### Librairie mlxtend

In [1]:
import numpy as np
from mlxtend.evaluate import lift_score

y_target =    np.array([0, 0, 1, 0, 0, 1, 1, 1, 1, 1])
y_predicted = np.array([1, 0, 1, 0, 0, 0, 0, 1, 0, 0])

lift_score(y_target, y_predicted)

1.1111111111111112

### Using lift_score in GridSearch

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import make_scorer

# make custom scorer
lift_scorer = make_scorer(lift_score)

iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=123)

hyperparameters = [{'kernel': ['rbf'],
                    'gamma': [1e-3, 1e-4],
                    'C': [1, 10, 100, 1000]}
                  ]

clf = GridSearchCV(SVC(), hyperparameters, cv=10,  scoring=lift_scorer)
clf.fit(X_train, y_train)

print(clf.best_score_)
print(clf.best_params_)

## Ressources :
 - [Lift Analysis – A Data Scientist’s Secret Weapon](https://www.kdnuggets.com/2016/03/lift-analysis-data-scientist-secret-weapon.html)
 - [Lift Score](https://rasbt.github.io/mlxtend/user_guide/evaluate/lift_score/)
 - [The ultimate guide to binary classification metrics](https://towardsdatascience.com/the-ultimate-guide-to-binary-classification-metrics-c25c3627dd0a#248c)
 - [Understand Gain and Lift Charts](https://www.listendata.com/2014/08/excel-template-gain-and-lift-charts.html)
 - [11 Important Model Evaluation Metrics for Machine Learning Everyone should know](https://www.analyticsvidhya.com/blog/2019/08/11-important-model-evaluation-error-metrics/)
 - [MLxtend](https://rasbt.github.io/mlxtend/)